#  Model Test


## 3.1 Capacity Model Accuracy Test

### 3.1.1 Generate test day selection

#### Input：

    od.csv
    
#### Output：

    od_test_10days.csv

Randomly sample 10 working days from od.csv and generate od_test_10days.csv

In [ ]:
import pandas as pd
import numpy as np

# Read od.csv
df = pd.read_csv("od.csv")

# Select only working days (day_type = 1)
df_work = df[df["day_type"] == 1]

# Get all unique working-day dates
unique_days = df_work["date"].unique()

# Fix random seed
np.random.seed(10)

# Randomly select 10 working days
sample_days = np.random.choice(unique_days, 10, replace=False)

print("Ten randomly sampled working days：")
print(sample_days)

# Extract corresponding OD records
df_test = df_work[df_work["date"].isin(sample_days)]

# Export to a new file
df_test.to_csv("od_test_10days.csv", index=False)

print("Output od_test_10days.csv")

Ten randomly sampled working days：
['2020-12-14' '2018-12-19' '2019-12-18' '2020-04-07' '2021-01-29'
 '2020-04-08' '2019-06-25' '2019-05-14' '2021-01-07' '2019-11-14']
Output od_test_10days.csv


### 3.1.2 Capacity Model Test

#### Input：
    od_test_10days.csv
    Minlimit_capacity_daytype_1.csv
    
  


  

In [ ]:
import pandas as pd
from collections import defaultdict

# Read input data
df_od = pd.read_csv("od_test_10days.csv")
df_model = pd.read_csv("Minlimit_capacity_daytype_1.csv")  # station_id, x_i, y_i

# Build dictionaries for initial bikes and capacity
x = dict(zip(df_model.station_id, df_model.x_i))
y = dict(zip(df_model.station_id, df_model.y_i))

#  Loop over each test day 
for day in df_od["date"].unique():
    df_day = df_od[df_od["date"] == day].sort_values(by=["hour"])

    # Initialize inventory for each station
    inventory = {st: x[st] for st in x}

    # Violation counters
    stockout = defaultdict(int)
    overflow = defaultdict(int)

    # count demand (departures + arrivals, used for ratio calculation)
    demand_count = defaultdict(int)

    for _, row in df_day.iterrows():
        s = row["start_station_id"]
        t = row["end_station_id"]
        k = row["trips"]

        # Record demand 
        demand_count[s] += k
        demand_count[t] += k

        # Borrow bikes — check for stockout
        inventory[s] -= k
        if inventory[s] < 0:
            stockout[s] += 1    

        # Return bikes — check for overflow
        inventory[t] += k
        if inventory[t] > y[t]:
            overflow[t] += 1    

    # Output results for the day
    print(f"{day} :")

    total_ratio_sum = 0  #Sum of violation ratios


    for station in x.keys():
        total_violation = stockout[station] + overflow[station]
        total_demand = demand_count[station]

        # Avoid division by zero
        ratio = (total_violation / total_demand)/2 if total_demand > 0 else 0

        total_ratio_sum += ratio


    print(f"violation total ratio: {total_ratio_sum:.4f} %")

2018-12-19 :
violation total ratio: 5.2843 %
2019-05-14 :
violation total ratio: 10.3224 %
2019-06-25 :
violation total ratio: 6.9034 %
2019-11-14 :
violation total ratio: 9.6028 %
2019-12-18 :
violation total ratio: 10.4060 %
2020-04-07 :
violation total ratio: 7.6116 %
2020-04-08 :
violation total ratio: 10.9951 %
2020-12-14 :
violation total ratio: 6.7997 %
2021-01-07 :
violation total ratio: 5.9583 %
2021-01-29 :
violation total ratio: 6.6475 %


### 3.2 Price test

Firstly, compute daily net revenue under fixed demand given ticket mix, prices, and cost terms.

Then, function splits total demand into single, day, and monthly passes, calculates user and corporate revenue, subtracts capacity and rebalancing costs, and returns a one-row DataFrame.

Finally, subtract the minimized costs of the two models to obtain the objective function for the total daily expenditure.


In [1]:
import pandas as pd

def compute_daily_net_expnditure(
    N_daily,                  # Total daily trips
    share_single,             # Share of single-trip tickets
    share_day,                # Share of day passes
    share_month,              # Share of monthly passes
    p_single,                 # Single-trip price (£)
    p_day,                    # Day-pass price (£)
    p_month,                  # Monthly-pass price (£)
    theta=0.0,                # Corporate subsidy share for monthly pass (0 = none, 1 = fully covered)
    Z1_capacity=0.0,          # Daily cost from capacity model
    Z2_rebalance=0.0,         # Daily cost from rebalancing model
):


    # Ticket quantities under fixed demand
    D_single = N_daily * share_single
    D_day    = N_daily * share_day
    D_month  = N_daily * share_month

    # Revenue

    # User-paid part
    R_user = (
        D_single * p_single +
        D_day * p_day +
        D_month * (0.85 - theta) * p_month
    )

    # Corporate subsidy part
    R_corp = D_month * theta * p_month

    # Total revenue
    R_total = R_user + R_corp

    # Total cost
    Total_cost = Z1_capacity + Z2_rebalance 

    # Net pexpenditure
    Net_expenditure =  Total_cost - R_total 

    result = {
        "Daily_Demand": N_daily,
        "D_single": D_single,
        "D_day": D_day,
        "D_month": D_month,
        "Revenue_user": R_user,
        "Revenue_corporate": R_corp,
        "Revenue_total": R_total,
        "Cost_capacity": Z1_capacity,
        "Cost_rebalance": Z2_rebalance,
        "Expenditure_daily":  Net_expenditure
    }

    return pd.DataFrame([result])


# Run
if __name__ == "__main__":
    N = 349                   # Daily trips
    shares = (0.55, 0.25, 0.20)  # 55% single, 25% day pass, 20% monthly pass
    prices = (1.65, 3.5, 20)     # Ticket prices (£)
    theta = 0.4                # Corporate covers 40% of monthly pass price
    Z1 =  1546.26               # Daily capacity cost (copied from model result)
    Z2 = 106.98                # Daily rebalancing cost
   
    df = compute_daily_net_expnditure(
        N_daily = N,
        share_single = shares[0],
        share_day    = shares[1],
        share_month  = shares[2],
        p_single = prices[0],
        p_day    = prices[1],
        p_month  = prices[2],
        theta = theta,
        Z1_capacity = Z1,
        Z2_rebalance = Z2,
    )

    print(df)

   Daily_Demand  D_single  D_day  D_month  Revenue_user  Revenue_corporate  \
0           349    191.95  87.25     69.8     1250.2925              558.4   

   Revenue_total  Cost_capacity  Cost_rebalance  Expenditure_daily  
0      1808.6925        1546.26          106.98          -155.4525  
